In [1]:
# Global Variables Set by User -- change these values before running script

# URL of the portal with the template content
PROJECT_URL = "https://envisioning.maps.arcgis.com"

# Portal Username for Log-on (needs to be admin)
PROJECT_USERNAME = "GBushongENV"

# URL of the portal that will be customized and have users added
DELIVERY_URL = "https://esrienergy.maps.arcgis.com"

# Delivery username for log-on (needs to be admin)
DELIVERY_USERNAME = "PDOAdmin"

# delivery groups to share with *ALL* users
GROUPS = []

# featured group to display on delivery home page
FEATURED_GROUP = "aec-test-group-2"

# delivery org UX Component locations - local full path
FOLDER = "/Users/grac9792/Downloads/Sample_Config" 

# filenames of UI components and user file, which are located in FOLDER
THUMBNAIL_FILENAME = "png-thumbnail-2.png"
FOOTER_FILENAME = "footer.txt"
BACKGROUND_FILENAME = "background.png"
BANNER_FILENAME = "banner.jpg"
DESCRIPTION_FILENAME = "description.txt"
USERS_FILENAME = "users.csv"

# message that will be sent to all users when they receive the invitations to join the group
MESSAGE = "You are invited to join a group!"

# how long the invites will be valid for: either '1 Day', '3 Days', '1 Week', or '2 Weeks'
EXPIRATION = '1 Day'

In [2]:
# import libraries
from arcgis.gis import GIS
import os

In [3]:
# get UI component filepaths
THUMBNAIL = os.path.join(FOLDER, THUMBNAIL_FILENAME)
FOOTER = os.path.join(FOLDER, FOOTER_FILENAME)
BACKGROUND = os.path.join(FOLDER, BACKGROUND_FILENAME)
BANNER = os.path.join(FOLDER, BANNER_FILENAME)
DESCRIPTION = os.path.join(FOLDER, DESCRIPTION_FILENAME)
USERS = os.path.join(FOLDER, USERS_FILENAME)

In [3]:
# connect to project and delivery orgs
project = GIS(PROJECT_URL, PROJECT_USERNAME)
delivery = GIS(DELIVERY_URL, DELIVERY_USERNAME)

Enter password: ········
Enter password: ········


In [4]:
# Creation of functions to be used later
# Code from user_utils.py
"""utility functions to assist with adding users"""
import csv
from arcgis.gis import UserManager

USER_FIELDS = ["username", "password", "firstname", "lastname", "email", "role", "groups", "level"
              "description", "provider", "idp_username"]

def optional_args_handler(key, user, default=None):
    var = None
    if key in user:
        var = user[key]
    else:
        if default: var = default
    return var

def add_user(user, origin, target, groups=None):
    """Add user to the gis
    * Abstraction for creating from dict such as with csv
    * Handles moving to groups

    args:
    user -- a dictionary containing user fields, see fields:
        http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.UserManager.create
    origin -- the org that the groups for the user to be added to exist
    target -- gis object where users are added
    groups -- (optional) destination groups, compliments those in dict (default [])
    field_map -- (optional) change keys from defaults in USER_FIELDS to those in dict
    """

    # set defaults
    groups = groups if groups else []

    try:
        print("INFO: Creating user {}".format(user["username"]))
        desc = optional_args_handler('description', user)
        thumbnail = optional_args_handler('thumbnail', user)
        role = optional_args_handler('role', user, 'org_user') # default is org_user
        provider = optional_args_handler('provider', user, 'arcgis') # default is 'arcgis'
        idp = optional_args_handler('idp_username', user)
        level = optional_args_handler('level', user, '2') # default is 2
        
        if role not in ['org_user', 'org_publisher', 'org_admin', 'org_viewer']:
            role = 'org_user'
                        
        # create array of groups for user
        group_str = user.pop('groups', None)
        if group_str:
            group_list = group_str.split(",")
            for g in group_list:
                group_search = origin.groups.search(g)
                if group_search:
                    groups.append(group_search[0])
        print(groups)
        
        result = target.users.create(username=user['username'], password=user['password'], firstname=user['firstname'], 
                                  lastname=user['lastname'], email=user['email'], description=desc,
                                  role=role, provider=provider, idp_username=idp, level=level, thumbnail=thumbnail)

        # create users returns None if it was unsuccessful
        if not result:
            print("Did not create user: Check username {}".format(user['username']))
            return

        # Add user to groups
        print("adding to groups")
        for g in groups:
            try:
                res = g.invite_by_email(user['email'], '{} {},\n{}}'.format(user['firstname'], user['lastname'], MESSAGE), expiration=EXPIRATION)
                print(res)
            except Exception as e:
                print("ERR: Could not add user {} to group {}: {}".format(user['username'], g, e))
        return result

    except Exception as e:
        print("ERR: Could not create user {}: {}".format(user['username'], e))

def add_users_csv(csv_file, origin, target, groups=None):
    """Add users from csv to gis
    * Convenient abstraction for csvs

    args:
    csv_file -- path to csv with users to create
    gis -- gis object where users are added
    groups -- (optional) destination groups, compliments those in csv (default [])
    field_map -- (optional) change keys from defaults in USER_FIELDS to those in csv, see fields:
    http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.UserManager.create
    """
    results = []
    with open(csv_file, 'r') as users_csv:
        users = csv.DictReader(users_csv)
        for user in users:
            result = add_user(user, origin, target, groups=groups)
            results.append(result)
    return results

In [5]:
# fetch groups where *ALL* users will be added
share_groups = [gis.groups.search(g)[0] for g in GROUPS]

# add users to the delivery org and invite them to groups in project org
add_users_csv(USERS, project, delivery, groups=share_groups)

INFO: Creating user ljccstudent1
[<Group title:"aec-test-group" owner:GBushongENV>]


Unable to create ljccstudent1


Unable to create ljccstudent1
Did not create user: Check username ljccstudent1
INFO: Creating user ljccstudent2
[<Group title:"aec-test-group" owner:GBushongENV>, <Group title:"aec-test-group-2" owner:GBushongENV>]


Unable to create ljccstudent2


Unable to create ljccstudent2
Did not create user: Check username ljccstudent2
INFO: Creating user ljccstudent3
[<Group title:"aec-test-group-2" owner:GBushongENV>]


Unable to create ljccstudent3


Unable to create ljccstudent3
Did not create user: Check username ljccstudent3
INFO: Creating user ljccstudent4
[<Group title:"aec-test-group" owner:GBushongENV>, <Group title:"aec-test-group-2" owner:GBushongENV>]


Unable to create ljccstudent4


Unable to create ljccstudent4
Did not create user: Check username ljccstudent4


[None, None, None, None]

In [7]:
# customize UI by setting background, banner, logo, etc.
ux = delivery.admin.ux
ux.set_background(BACKGROUND)
ux.set_banner(BANNER)
ux.set_logo(THUMBNAIL)

# read description text file and set portal description
with open(DESCRIPTION, 'r') as f:
    description = f.read()
    ux.description = description
    
# find and set featured group
featured_group = delivery.groups.search(FEATURED_GROUP)[0]
ux.featured_content = {'group': featured_group}